# Day 5 - Community Contribution by Hayatu

**Global Business Compliance Assistant** — Enhancing Day 1 with a senior business compliance consultant persona.

This notebook takes a country and business type, then generates a structured compliance checklist. It optionally scrapes official regulator websites for up-to-date context.

In [ ]:
# imports
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join("..", "..")))

from dotenv import load_dotenv
from scraper import fetch_website_contents
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

MODEL = "gpt-4o-mini"
openai = OpenAI()

## Senior Business Compliance Consultant — System Prompt

The persona guides the model to produce structured, country-specific compliance advice.

In [ ]:
COMPLIANCE_SYSTEM_PROMPT = """
You are a senior business compliance consultant with 20+ years of experience helping entrepreneurs start and register businesses in over 50 countries. You know regulatory frameworks, licensing agencies, tax obligations, and common pitfalls for SMEs.

Your style:
- Clear and structured — use numbered lists, headings, and short paragraphs
- Practical — focus on what to do, in what order, and why
- Cautious — you always advise users to verify with a local lawyer or accountant for their specific case
- Concise — no fluff; every point should be actionable

You tailor your advice to the user's country and business type. You cite real agencies (e.g. CAC, FIRS, NAFDAC for Nigeria; Companies House, HMRC for UK; IRS, state agencies for USA) and use correct local terminology.

You never invent regulation; if you're uncertain, you say so and recommend official sources or professional verification.
"""

## Compliance Assistant — Core Function

Accepts `country` and `business_type`. Optionally pass `regulator_url` to scrape an official site for additional context.

In [ ]:
def messages_for_compliance(country: str, business_type: str, regulator_url: str | None = None) -> list[dict]:
    user_content = f"""
Country: {country}
Business type: {business_type}

Please provide a structured compliance checklist covering:
- Registration requirements (what agencies to register with, in what order)
- Licenses and permits (industry-specific)
- Tax obligations (VAT/GST, income tax, etc.)
- Common mistakes that cause rejections or fines
- Estimated timelines and costs where you know them

Respond in markdown. Do not wrap the markdown in a code block.
"""
    if regulator_url:
        try:
            scraped = fetch_website_contents(regulator_url)
            user_content += f"\n\n---\nAdditional context from official source ({regulator_url}):\n\n{scraped[:3000]}"
        except Exception as e:
            user_content += f"\n\n(Note: Could not fetch {regulator_url}: {e})"

    return [
        {"role": "system", "content": COMPLIANCE_SYSTEM_PROMPT},
        {"role": "user", "content": user_content}
    ]


def get_compliance_advice(country: str, business_type: str, regulator_url: str | None = None) -> str:
    messages = messages_for_compliance(country, business_type, regulator_url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages
    )
    return response.choices[0].message.content


def display_compliance(country: str, business_type: str, regulator_url: str | None = None):
    result = get_compliance_advice(country, business_type, regulator_url)
    display(Markdown(result))

## Streaming Compliance Report

Same as above, but streams the response for a typewriter effect.

In [ ]:
def stream_compliance(country: str, business_type: str, regulator_url: str | None = None):
    messages = messages_for_compliance(country, business_type, regulator_url)
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response), display_id=display_handle.display_id)

## Test Inputs — 3 Countries from Different Continents

**1. Nigeria (Africa)** — Food processing  
**2. United Kingdom (Europe)** — Online tutoring  
**3. Brazil (South America)** — Electronics import/e-commerce

In [ ]:
# Test 1: Nigeria (Africa) — food processing
stream_compliance(
    country="Nigeria",
    business_type="I want to start a small business packaging and selling chin-chin (fried snacks) to supermarkets and schools. I'll produce from a home kitchen initially."
)

In [ ]:
# Test 2: United Kingdom (Europe) — online tutoring
stream_compliance(
    country="United Kingdom",
    business_type="I want to run an online tutoring business offering math and English lessons to school-age children. I'll work from home as a sole trader."
)

In [ ]:
# Test 3: Brazil (South America) — electronics import/e-commerce
stream_compliance(
    country="Brazil",
    business_type="I want to import and sell electronics (phones, accessories) through an e-commerce store. I'll operate as a small company."
)

## Optional: Include Scraped Regulator Content

Pass a `regulator_url` to fetch and include official agency website content for more accurate, up-to-date advice.

In [ ]:
# Example: Nigeria CAC (Corporate Affairs Commission) for registration context

stream_compliance(
    country="Nigeria",
    business_type="I want to register a business name for a consulting firm.",
    regulator_url="https://www.cac.gov.ng"
)